In [1]:
import numpy as np
import pandas as pd

数据预处理与logistic回归中是一样的

In [2]:
headers = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    'income'
]
train_data = pd.read_csv('adult.data',header=None)
test_data = pd.read_csv('adult.test',header=None)
train_data.columns = headers
test_data.columns = headers
income_map = {
    ' >50K': 1,
    ' <=50K': 0
}
train_data['income'] = train_data['income'].map(income_map)
income_map = {
   ' >50K.': 1,
   ' <=50K.': 0
}
test_data['income'] = test_data['income'].map(income_map)
# 筛选出类型为 object 的列
object_columns = train_data.select_dtypes(include=['object']).columns
# 对这些列应用独热编码
train_data = pd.get_dummies(train_data, columns=object_columns)
test_data = pd.get_dummies(test_data,columns=object_columns)
new_column = pd.Series(False, index=test_data.index)
new_column.name = 'native-country_ Holand-Netherlands'
test_data.insert(82,new_column.name,new_column)
X_train,y_train = train_data.drop(columns='income').to_numpy().astype(np.float64),train_data['income'].to_numpy().astype(np.float64)
X_test,y_test = test_data.drop(columns='income').to_numpy().astype(np.float64),test_data['income'].to_numpy().astype(np.float64)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [3]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(32561, 108) (32561, 1)
(16281, 108) (16281, 1)


模型定义

In [4]:
def likelihood(X,mean,var):
    ep = 1e-10
    res = 1/((np.sqrt(2*np.pi*var)+ep))*np.exp(-((X-mean)**2)/(2*var**2+ep))
    return np.clip(res,ep,1-ep)
def naive(X,y):
    y = y.reshape(-1)
    y_type = np.unique(y)
    m = len(y_type)
    # 均值及方差
    means = np.zeros((m,X.shape[1]))
    var = np.zeros((m,X.shape[1]))
    # 先验概率
    priors = np.zeros(m)

    for i,label in enumerate(y_type):
        # 获取对应类的所有样本
        Xi = X[y==label]
        # 先计算均值和方差
        means[i] = np.mean(Xi,axis=0)
        var[i] = np.mean(Xi,axis=0)
        # 计算先验概率
        priors[i] = len(Xi)/len(X)
    return means,var,priors,y_type
# 预测函数
def predict(X,means,var,priors,y_type):
    y_ = []
    for i,label in enumerate(y_type):
        prior = np.log(priors[i])  # 获取先验概率
        likelihood_i = np.sum(np.log(likelihood(X, means[i], var[i])), axis=1)
        posterior = prior + likelihood_i  # 计算后验概率
        y_.append(posterior)

    y_ = np.array(y_).T
    y_hat = y_type[np.argmax(y_, axis=1)]  # 根据计算出的概率，进行分类
    return y_hat

得到朴素贝叶斯模型的参数

In [5]:
means,var,priors,y_type = naive(X_train,y_train)

计算准确率

In [6]:
y_pred = predict(X_test, means, var, priors, y_type)
accuracy = np.mean(y_pred == y_test)
print(f'准确率:{accuracy}')

准确率:0.6144947427870079


In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_curve, auc

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

print(f"ROC-AUC 值: {roc_auc}")
print(f'精确率: {precision}')
print(f'召回率: {recall}')

ROC-AUC 值: 0.7309306678660392
精确率: 0.5307141306707185
召回率: 0.6357254290171607
